In [3]:
import requests, re, pandas as pd
import time
from bs4 import BeautifulSoup

STATENAME = 'California'
urls = pd.read_csv(STATENAME+'_urls.csv')

hotel_list = []
pos_review_title_list = []
pos_review_content_list = []
neg_review_title_list = []
neg_review_content_list = []
overall_rate_list = []
review_num_list = []

for baseurl in urls['url']:
    
    pos_review_title =''
    pos_review_content =''
    neg_review_title =''
    neg_review_content =''
    overall_rate = float("nan")
    review_num = float("nan")
    
    if type(baseurl) == str and ('Hotel_Review' in baseurl):
        hotel = 1
    else:
        hotel = 0
    
    
    for review_index in range(0, 11, 10):
        if review_index == 0:
            url = baseurl
        else:
            url = baseurl.replace('-Reviews-', '-Reviews-or'+str(review_index)+'-')
    
        try:
            headers = {
                'User-Agent':  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
            }
            html = requests.get(url, headers=headers).text
            soup = BeautifulSoup(html)

            scores = []
            
            if hotel == False:
                for bubble in soup.body.main.find_all('div', class_='LbPSX')[0].find_all('svg',class_='UctUV d H0'):
                    regex = r"(\d+\.\d+) of 5 bubbles"
                    match = re.search(regex, str(bubble))
                    if match != None:
                        scores.append(float(match.group(1)))
            
                reviews = soup.body.main.find_all('span',class_='yCeTE')

                for i in range(len(scores)):
                    if scores[i]>=4.0:
                        pos_review_title = pos_review_title + '. ' + str(reviews[2*i].string)
                        pos_review_content = pos_review_content + '. ' + str(reviews[2*i+1].string)
                    else:
                        neg_review_title = neg_review_title + '. ' + str(reviews[2*i].string)
                        neg_review_content = neg_review_content + '. ' + str(reviews[2*i+1].string)

                regex = r"(\d+\.\d+) of 5 bubbles. (\d+) reviews"
                match = re.search(regex, str(soup.body.main.find_all('div',class_='jVDab o W f u w GOdjs')[0]).replace(',',''))
                if match != None and review_index == 0:
                    overall_rate = float(match.group(1))
                    review_num = float(match.group(2))
                    print(hotel, url, overall_rate, review_num)
            
            else:
                
                for bubble in soup.body.find_all('div', class_='Hlmiy F1'):
                    regex = r"\d+"
                    match = re.search(regex, str(bubble.span))
                    if match != None:
                        scores.append(float(match.group(0))/10)
                
                reviews_title = soup.body.find_all('div', class_='KgQgP MC _S b S6 H5 _a')
                reviews_content = soup.body.find_all('q', class_='QewHA H4 _a')

                for i in range(len(scores)):
                    if scores[i]>=4.0:
                        pos_review_title = pos_review_title + '. ' + str(reviews_title[i].find_all('span')[0].span.string)
                        pos_review_content = pos_review_content + '. ' + str(reviews_content[i].string)
                    else:
                        neg_review_title = neg_review_title + '. ' + str(reviews_title[i].find_all('span')[0].span.string)
                        neg_review_content = neg_review_content + '. ' + str(reviews_content[i].string)

                if review_index == 0:
                    overall_rate = float(soup.body.find_all('span', class_='uwJeR P')[0].string)
                    review_num = float(str(soup.body.find_all('span', class_='hkxYU q Wi z Wc')[0]).replace(',','').split('>')[1].split('<')[0])
                    print(hotel, url, overall_rate, review_num)
            
                
        except:
            break
    
    hotel_list.append(hotel)
    overall_rate_list.append(overall_rate)
    review_num_list.append(review_num)
    pos_review_title_list.append(pos_review_title)
    pos_review_content_list.append(pos_review_content)
    neg_review_title_list.append(neg_review_title)
    neg_review_content_list.append(neg_review_content)
    
    time.sleep(3)

    

doc_xue = pd.read_csv(STATENAME+'_documents.csv')
doc_xue['hotel'] = hotel_list
doc_xue['overall_rate'] = overall_rate_list
doc_xue['review_num'] = review_num_list
doc_xue['pos_review_title'] = pos_review_title_list
doc_xue['pos_review_content'] = pos_review_content_list
doc_xue['neg_review_title'] = neg_review_title_list
doc_xue['neg_review_content'] = neg_review_content_list
doc_xue[~doc_xue['review_num'].isna()].to_csv(STATENAME+'_final.csv', index = False)


0 https://www.tripadvisor.com/Attraction_Review-g60713-d144979-Reviews-1_5_mile_Coastal_Trail-San_Francisco_California.html 4.5 5.0
0 https://www.tripadvisor.com/Attraction_Review-g32588-d561800-Reviews-1_000_Steps_Beach-Laguna_Beach_California.html 4.5 342.0
0 https://www.tripadvisor.com/Attraction_Review-g32737-d146192-Reviews-17_Mile_Drive-Monterey_Monterey_County_California.html 4.5 12615.0
1 https://www.tripadvisor.com/Hotel_Review-g32363-d5994509-Reviews-Broadway_Motel-Eureka_Humboldt_County_California.html 3.0 3.0
0 https://www.tripadvisor.com/Attraction_Review-g33052-d110204-Reviews-26_Mile_Bike_Path-Santa_Monica_California.html 4.5 1480.0
0 https://www.tripadvisor.com/Attraction_Review-g32143-d286905-Reviews-Lee_Youngman_Galleries-Calistoga_Napa_Valley_California.html 4.0 9.0
0 https://www.tripadvisor.com/Attraction_Review-g60713-d143321-Reviews-450_Sutter_Building-San_Francisco_California.html 4.0 20.0
0 https://www.tripadvisor.com/Attraction_Review-g60713-d313399-Reviews-49_

0 https://www.tripadvisor.com/Attraction_Review-g32999-d142861-Reviews-American_River_Bicycle_Trail-Sacramento_California.html 4.5 625.0
0 https://www.tripadvisor.com/Attraction_Review-g32999-d128282-Reviews-American_River_Parkway-Sacramento_California.html 4.5 155.0
0 https://www.tripadvisor.com/Attraction_Review-g32840-d209504-Reviews-American_Tin_Cannery_Factory_Outlets-Pacific_Grove_Monterey_County_California.html 2.5 54.0
0 https://www.tripadvisor.com/Attraction_Review-g32761-d181822-Reviews-NASA_Ames_Visitor_Center-Mountain_View_California.html 3.5 174.0
0 https://www.tripadvisor.com/Attraction_Review-g143019-d108635-Reviews-Anacapa_Island-Channel_Islands_National_Park_California.html 4.5 104.0
0 https://www.tripadvisor.com/Attraction_Review-g29092-d280547-Reviews-Anaheim_Indoor_Marketplace-Anaheim_California.html 3.5 42.0
0 https://www.tripadvisor.com/Attraction_Review-g32400-d278804-Reviews-Ananse_Village-Fort_Bragg_Mendocino_County_California.html 4.5 29.0
0 https://www.tripad

1 https://www.tripadvisor.com/Hotel_Review-g33045-d81889-Reviews-Avania_Inn_of_Santa_Barbara-Santa_Barbara_California.html 4.5 208.0
1 https://www.tripadvisor.com/Hotel_Review-g32648-d1370955-Reviews-Hyatt_Centric_The_Pike_Long_Beach-Long_Beach_California.html 4.5 1530.0
1 https://www.tripadvisor.com/Hotel_Review-g32766-d1370957-Reviews-Andaz_Napa-Napa_Napa_Valley_California.html 4.5 2880.0
0 https://www.tripadvisor.com/Attraction_Review-g32655-d156640-Reviews-Avila_Adobe-Los_Angeles_California.html 4.0 109.0
1 https://www.tripadvisor.com/Hotel_Review-g29124-d607833-Reviews-Avila_Lighthouse_Suites-Avila_Beach_San_Luis_Obispo_County_California.html 4.5 1516.0
1 https://www.tripadvisor.com/Hotel_Review-g29124-d543141-Reviews-Avila_Village_Inn-Avila_Beach_San_Luis_Obispo_County_California.html 4.5 955.0
0 https://www.tripadvisor.com/Attraction_Review-g33231-d1454361-Reviews-Avo_Playhouse-Vista_California.html 4.5 16.0
1 https://www.tripadvisor.com/Hotel_Review-g32478-d285421-Reviews-Ayres

0 https://www.tripadvisor.com/Attraction_Review-g60713-d128979-Reviews-Bernal_Heights_Park-San_Francisco_California.html 4.5 60.0
1 https://www.tripadvisor.com/Hotel_Review-g32173-d261199-Reviews-Bernardus_Lodge_and_Spa-Carmel_Valley_Monterey_County_California.html 4.5 679.0
0 https://www.tripadvisor.com/Attraction_Review-g32513-d578944-Reviews-Best_Friend_Dog_Park-Huntington_Beach_California.html 4.5 45.0
1 https://www.tripadvisor.com/Hotel_Review-g32171-d76347-Reviews-Best_Western_Plus_Beach_View_Lodge-Carlsbad_California.html 4.5 935.0
1 https://www.tripadvisor.com/Hotel_Review-g33045-d501980-Reviews-Beachside_Inn-Santa_Barbara_California.html 4.0 120.0
1 https://www.tripadvisor.com/Hotel_Review-g60750-d79907-Reviews-Blue_Sea_Beach_Hotel-San_Diego_California.html 4.5 1889.0
1 https://www.tripadvisor.com/Hotel_Review-g32655-d84555-Reviews-The_Carlyle_Inn-Los_Angeles_California.html 4.0 746.0
1 https://www.tripadvisor.com/Hotel_Review-g32172-d76372-Reviews-Carmel_Bay_View_Inn-Carmel_M

1 https://www.tripadvisor.com/Hotel_Review-g32148-d113953-Reviews-Blue_Dolphin_Inn-Cambria_San_Luis_Obispo_County_California.html 4.5 1020.0
0 https://www.tripadvisor.com/Attraction_Review-g33252-d107068-Reviews-Blue_Jay_Way-West_Hollywood_California.html 3.0 5.0
0 https://www.tripadvisor.com/Attraction_Review-g32363-d599122-Reviews-Blue_Ox_Millworks_Historic_Park-Eureka_Humboldt_County_California.html 4.5 39.0
0 https://www.tripadvisor.com/Attraction_Review-g32684-d268999-Reviews-Bluewater_Sailing-Marina_del_Rey_California.html 4.5 17.0
0 https://www.tripadvisor.com/Attraction_Review-g32279-d5777049-Reviews-Bluff_Top_Trail-Dana_Point_California.html 4.5 52.0
0 https://www.tripadvisor.com/Attraction_Review-g32661-d208331-Reviews-Bluff_Trail-Los_Osos_San_Luis_Obispo_County_California.html 5.0 88.0
1 https://www.tripadvisor.com/Hotel_Review-g32123-d76254-Reviews-Los_Angeles_Marriott_Burbank_Airport-Burbank_California.html 4.0 1082.0
1 https://www.tripadvisor.com/Hotel_Review-g32091-d7616

...

1 https://www.tripadvisor.com/Hotel_Review-g32715-d80687-Reviews-The_Westin_San_Francisco_Airport-Millbrae_California.html 4.0 1101.0
1 https://www.tripadvisor.com/Hotel_Review-g32253-d76671-Reviews-The_Westin_South_Coast_Plaza_Costa_Mesa-Costa_Mesa_California.html 4.0 1029.0
1 https://www.tripadvisor.com/Hotel_Review-g60713-d81461-Reviews-The_Westin_St_Francis_San_Francisco_on_Union_Square-San_Francisco_California.html 4.0 944.0
1 https://www.tripadvisor.com/Hotel_Review-g32766-d1141009-Reviews-The_Westin_Verasa_Napa-Napa_Napa_Valley_California.html 4.0 1654.0
1 https://www.tripadvisor.com/Hotel_Review-g60713-d81466-Reviews-The_Wharf_Inn-San_Francisco_California.html 3.5 1066.0
1 https://www.tripadvisor.com/Hotel_Review-g32293-d536108-Reviews-Nurturing_Nest_Mineral_Hot_Springs_Retreat_and_Spa-Desert_Hot_Springs_Greater_Palm_Sprin.html 4.5 114.0
0 https://www.tripadvisor.com/Attraction_Review-g32837-d651306-Reviews-The_Woolworth_Building-Oxnard_California.html 4.0 38.0
1 https://www.tr

0 https://www.tripadvisor.com/Attraction_Review-g32132-d10838284-Reviews-Tule_Elk_Reserve_State_Natural_Reserve-Buttonwillow_California.html 4.0 8.0
0 https://www.tripadvisor.com/Attraction_Review-g61000-d593212-Reviews-Tunnel_View-Yosemite_National_Park_California.html 5.0 3071.0
0 https://www.tripadvisor.com/Attraction_Review-g33108-d116869-Reviews-Tuolumne_County_Museum-Sonora_California.html 4.5 36.0
1 https://www.tripadvisor.com/Hotel_Review-g61000-d119548-Reviews-Tuolumne_Meadows_Lodge-Yosemite_National_Park_California.html 4.5 230.0
0 https://www.tripadvisor.com/Attraction_Review-g32627-d215360-Reviews-Turkey_Creek_Golf_Club-Lincoln_California.html 4.5 41.0
0 https://www.tripadvisor.com/Attraction_Review-g32952-d265300-Reviews-Turtle_Bay_Exploration_Park-Redding_California.html 4.5 452.0
0 https://www.tripadvisor.com/Attraction_Review-g33198-d2225625-Reviews-Bowlmor_Lanes-Tustin_California.html 4.0 13.0
0 https://www.tripadvisor.com/Attraction_Review-g33198-d270729-Reviews-Tusti

0 https://www.tripadvisor.com/Attraction_Review-g60769-d538023-Reviews-Ventura_County_Museum_of_History_Art-Ventura_California.html 4.5 54.0
0 https://www.tripadvisor.com/Attraction_Review-g60769-d1645672-Reviews-Ventura_County_Wine_Trail-Ventura_California.html 5.0 6.0
0 https://www.tripadvisor.com/Attraction_Review-g32530-d7306901-Reviews-Xtreme_Adventures-Irvine_California.html 5.0 90.0
0 https://www.tripadvisor.com/Attraction_Review-g60713-d546392-Reviews-Vermont_Street-San_Francisco_California.html 4.5 19.0
0 https://www.tripadvisor.com/Attraction_Review-g61000-d483481-Reviews-Vernal_Fall-Yosemite_National_Park_California.html 4.5 1400.0
0 https://www.tripadvisor.com/Attraction_Review-g60713-d156726-Reviews-Vesuvio_Cafe-San_Francisco_California.html 4.5 282.0
0 https://www.tripadvisor.com/Attraction_Review-g33108-d1576521-Reviews-Veterans_Memorial_Hall_Military_Museum-Sonora_California.html 4.5 40.0
0 https://www.tripadvisor.com/Attraction_Review-g32285-d952225-Reviews-Viader_Vine

1 https://www.tripadvisor.com/Hotel_Review-g32655-d77700-Reviews-W_Los_Angeles_West_Beverly_Hills-Los_Angeles_California.html 4.0 988.0
0 https://www.tripadvisor.com/Attraction_Review-g60750-d104121-Reviews-Westfield_Horton_Plaza-San_Diego_California.html 3.0 379.0
1 https://www.tripadvisor.com/Hotel_Review-g60750-d218780-Reviews-The_Westin_San_Diego_Gaslamp_Quarter-San_Diego_California.html 4.5 212.0
1 https://www.tripadvisor.com/Hotel_Review-g32942-d543502-Reviews-The_Westin_Mission_Hills_Resort_Villas_Palm_Springs-Rancho_Mirage_Greater_Palm_Springs_C.html 4.0 1371.0
1 https://www.tripadvisor.com/Hotel_Review-g32942-d79405-Reviews-The_Westin_Rancho_Mirage_Golf_Resort_Spa-Rancho_Mirage_Greater_Palm_Springs_California.html 4.5 252.0
1 https://www.tripadvisor.com/Hotel_Review-g32942-d543502-Reviews-The_Westin_Mission_Hills_Resort_Villas_Palm_Springs-Rancho_Mirage_Greater_Palm_Springs_C.html 4.0 1371.0
1 https://www.tripadvisor.com/Hotel_Review-g60791-d673245-Reviews-The_Westin_Monache_R

1 https://www.tripadvisor.com/Hotel_Review-g61000-d119523-Reviews-Yosemite_Valley_Lodge-Yosemite_National_Park_California.html 4.0 4614.0
0 https://www.tripadvisor.com/Attraction_Review-g32385-d262477-Reviews-Yosemite_Mountain_Sugar_Pine_Railroad-Fish_Camp_California.html 4.5 735.0
0 https://www.tripadvisor.com/Attraction_Review-g61000-d146475-Reviews-Yosemite_Mountaineering_School-Yosemite_National_Park_California.html 4.5 59.0
0 https://www.tripadvisor.com/Attraction_Review-g61000-d1148875-Reviews-Yosemite_Mule_Horseback_Rides-Yosemite_National_Park_California.html 4.0 75.0
0 https://www.tripadvisor.com/Attraction_Review-g61000-d591079-Reviews-Yosemite_Museum_Gallery-Yosemite_National_Park_California.html 4.5 57.0
0 https://www.tripadvisor.com/Attraction_Review-g32385-d1101398-Reviews-Yosemite_Trails_Horseback_Adventures-Fish_Camp_California.html 4.5 191.0
0 https://www.tripadvisor.com/Attraction_Review-g61000-d531831-Reviews-Yosemite_Village-Yosemite_National_Park_California.html 4.

In [2]:
doc_xue = pd.read_csv(STATENAME+'_documents.csv')
doc_xue['hotel'] = hotel_list
doc_xue['overall_rate'] = overall_rate_list
doc_xue['review_num'] = review_num_list
doc_xue['pos_review_title'] = pos_review_title_list
doc_xue['pos_review_content'] = pos_review_content_list
doc_xue['neg_review_title'] = neg_review_title_list
doc_xue['neg_review_content'] = neg_review_content_list
doc_xue[~doc_xue['review_num'].isna()].to_csv(STATENAME+'_final.csv', index = False)